In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

import json

import pydicom

In [17]:
for patient in patients:
    print("Processing {}".format(patient))
    test = os.path.join(folder,patient)
    files = os.listdir(test)
    modality_count = {}
    unique_studies = []
    for file in files:
        temp = pydicom.read_file(os.path.join(test,file))
        m = temp.Modality
        if m not in modality_count.keys():
            modality_count[m] = 1
        else:
            modality_count[m] += 1
        unique_studies.append(temp.StudyInstanceUID)
    unique_studies = list(set(unique_studies))
    print(modality_count)
    print("Number of studies: {}".format(len(unique_studies)))

Processing ANON_001
{'CT': 528, 'RTDOSE': 4, 'REG': 1, 'RTIMAGE': 5, 'RTSTRUCT': 2}
Number of studies: 2
Processing ANON_002
{'CT': 185, 'RTDOSE': 4, 'RTIMAGE': 4, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_004
{'CT': 191, 'RTDOSE': 18, 'REG': 1, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_006
{'CT': 185, 'RTDOSE': 14, 'RTSTRUCT': 2}
Number of studies: 1
Processing ANON_007
{'CT': 193, 'RTDOSE': 6, 'RTIMAGE': 2, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_008
{'CT': 189, 'RTDOSE': 6, 'RTIMAGE': 4, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_009
{'CT': 185, 'RTDOSE': 4, 'REG': 1, 'RTIMAGE': 2, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_011
{'CT': 189, 'RTDOSE': 8, 'RTIMAGE': 2, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_013
{'CT': 189, 'RTDOSE': 6, 'REG': 1, 'RTIMAGE': 2, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_014
{'CT': 193, 'RTDOSE': 6, 'REG': 1, 'RTIMAGE': 3, 'RTSTRUCT': 1}
Number of studies: 1
Processing ANON_016
{'CT':

In [18]:
pt = os.path.join(folder, 'ANON_034')
files = os.listdir(pt)
dosefiles = []
for file in files:
    if file.startswith("RD"):
        dosefiles.append(pydicom.read_file(os.path.join(pt,file)))

In [25]:
for file in dosefiles:
    print("Series:",file.SeriesInstanceUID)
    print("FOR:",file.FrameOfReferenceUID)

Series: 1.2.246.352.221.48013794216801740736714951329911434929
FOR: 1.2.246.352.221.56292222622455867658124158751083335860
Series: 1.2.246.352.221.51832360988287704771798216858716028594
FOR: 1.2.246.352.221.538199235522034619012241948574345232829
Series: 1.2.246.352.221.48013794216801740736714951329911434929
FOR: 1.2.246.352.221.56292222622455867658124158751083335860
Series: 1.2.246.352.221.51832360988287704771798216858716028594
FOR: 1.2.246.352.221.538199235522034619012241948574345232829
Series: 1.2.246.352.221.48013794216801740736714951329911434929
FOR: 1.2.246.352.221.56292222622455867658124158751083335860
Series: 1.2.246.352.221.51832360988287704771798216858716028594
FOR: 1.2.246.352.221.538199235522034619012241948574345232829
Series: 1.2.246.352.221.475011064498961075915816881981683521950
FOR: 1.2.246.352.221.538199235522034619012241948574345232829
Series: 1.2.246.352.221.54939104244368711044505002034775700414
FOR: 1.2.246.352.221.538199235522034619012241948574345232829
Series: 1.

In [27]:
FORlist = []
for file in files:
    if not file.startswith("CT"):
        continue
    temp = pydicom.read_file(os.path.join(pt,file))
    try:
        FORlist.append(temp.FrameOfReferenceUID)
    except AttributeError:
        print("No FOR for {}".format(file))
FORlist = list(set(FORlist))
print(len(FORlist))

1


## Examining max dose for all data

In [1]:
rootfolder = r"D:\extracteddata"

In [4]:
contents = os.listdir(rootfolder)
patients = [x for x in contents if os.path.isdir(os.path.join(rootfolder,x))]

In [20]:
labels = pd.read_csv(r"D:\extracteddata\labels.csv")
labels = labels.set_index('patient')
labels = labels.to_dict()
for patient in patients:
    subfolder = os.path.join(rootfolder,patient)
    dose = np.load(os.path.join(subfolder,"dose.npy"))
    if patient in labels['xero'].keys():
        label = labels['xero'][patient]
    else:
        label = "unknown"
    print(patient,"- Max dose:",np.amax(dose),"-",label)

018_001 - Max dose: 75.71465232216107 - unknown
018_002 - Max dose: 75.10887975238363 - unknown
018_003 - Max dose: 75.63097850576179 - 1
018_005 - Max dose: 73.62690074691653 - 0
018_006 - Max dose: 78.17017702540997 - 0
018_007 - Max dose: 75.73280491360987 - unknown
018_008 - Max dose: 76.2838038270751 - unknown
018_009 - Max dose: 77.90929410901052 - 1
018_010 - Max dose: 75.01200316157207 - unknown
018_011 - Max dose: 1115524.5625 - unknown
018_012 - Max dose: 75.88907035669504 - 1
018_013 - Max dose: 78.26770952939951 - 1
018_014 - Max dose: 75.53036162919975 - 1
018_015 - Max dose: 77.72516297464583 - unknown
018_016 - Max dose: 76.82310574276246 - 0
018_018 - Max dose: 76.00933184989997 - 1
018_019 - Max dose: 1106743.5625 - 1
018_020 - Max dose: 74.75466923355678 - unknown
018_021 - Max dose: 75.21481900456986 - unknown
018_022 - Max dose: 78.58898843211938 - 0
018_023 - Max dose: 75.4455087349523 - unknown
018_024 - Max dose: 76.50249524810746 - 0
018_025 - Max dose: 77.48006